In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

class KriptoHaberScraper:
    def _init_(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')  # Arka planda çalışsın
        self.driver = webdriver.Chrome(options=self.options)
        
        # CSV dosyası oluştur ve başlıkları yaz
        self.csv_file = "doviz_kripto_haberler1-10.csv"
        with open(self.csv_file, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Başlık", "İçerik", "Tarih"])  # Tarih sütunu eklendi
        
    def sayfa_ac(self, url):
        self.driver.get(url)
        time.sleep(2)
        
    def sayfa_sayisi_al(self):
        try:
            pagination = self.driver.find_elements(By.CSS_SELECTOR, "ul.pagination li a")
            max_sayfa = 1
            
            for page in pagination:
                try:
                    sayfa_no = int(page.get_attribute('title').split()[-1])
                    if sayfa_no > max_sayfa:
                        max_sayfa = sayfa_no
                except:
                    continue
                    
            return min(max_sayfa, 90)  # Maximum 90 sayfa
        except Exception as e:
            print(f"Sayfa sayısı alınırken hata: {str(e)}")
            return 1
            
    def haber_linkleri_topla(self, sayfa_no):
        try:
            url = f"https://haber.doviz.com/kripto-para-haberleri?page={sayfa_no}"
            self.driver.get(url)
            time.sleep(2)
            
            haber_linkler = self.driver.find_elements(By.CSS_SELECTOR, "div.content-card")
            urls = []
            
            for haber in haber_linkler:
                try:
                    link = haber.find_element(By.XPATH, "./parent::a").get_attribute('href')
                    if link and link not in urls:  # Tekrar eden linkleri alma
                        urls.append(link)
                except:
                    continue
            
            return urls
            
        except Exception as e:
            print(f"Sayfa {sayfa_no} - Haber linkleri toplanırken hata: {str(e)}")
            return []
            
    def haber_icerigi_cek(self, url):
        try:
            self.driver.get(url)
            time.sleep(2)
            
            baslik = self.driver.find_element(By.TAG_NAME, "h1").text
            
            # Tarih bilgisi alınıyor
            try:
                tarih_raw = self.driver.find_element(By.CLASS_NAME, "publish-timestamp").text
                tarih = tarih_raw.split()[0]  # Sadece gün.ay.yıl kısmını al
            except:
                tarih = "Tarih bulunamadı"
            
            try:
                icerik_container = self.driver.find_element(By.CLASS_NAME, "content")
                paragraflar = icerik_container.find_elements(By.TAG_NAME, "p")
                icerik = "\n\n".join([p.text.strip() for p in paragraflar if p.text.strip()])
                icerik = ' '.join(icerik.split())  # Gereksiz boşlukları temizle
            except:
                icerik = "İçerik bulunamadı"
            
            if not icerik or icerik == "İçerik bulunamadı":
                return None
                
            return {
                'baslik': baslik,
                'icerik': icerik,
                'tarih': tarih
            }
            
        except Exception as e:
            print(f"Haber içeriği çekilirken hata: {str(e)}")
            return None
            
    def csv_kaydet(self, haber_data):
        try:
            with open(self.csv_file, mode='a', encoding='utf-8', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([haber_data['baslik'], haber_data['icerik'], haber_data['tarih']])  # Tarih bilgisi eklendi
            print(f"Haber kaydedildi: {haber_data['baslik']}")
        except Exception as e:
            print(f"CSV dosyasına kaydedilirken hata: {str(e)}")
            
    def calistir(self):
        try:
            ana_sayfa = "https://haber.doviz.com/kripto-para-haberleri"
            print("Kripto haberleri ana sayfası açılıyor...")
            self.sayfa_ac(ana_sayfa)
            
            toplam_sayfa = self.sayfa_sayisi_al()
            print(f"Toplam {toplam_sayfa} sayfa bulundu.")
            
            toplam_basarili = 0
            toplam_basarisiz = 0
            
            for sayfa_no in range(1, toplam_sayfa + 1):
                print(f"\nSayfa {sayfa_no}/{toplam_sayfa} işleniyor...")
                
                haber_linkleri = self.haber_linkleri_topla(sayfa_no)
                print(f"Sayfa {sayfa_no}: {len(haber_linkleri)} haber bulundu.")
                
                for i, link in enumerate(haber_linkleri, 1):
                    print(f"\nSayfa {sayfa_no} - Haber {i}/{len(haber_linkleri)} - İşleniyor: {link}")
                    haber_data = self.haber_icerigi_cek(link)
                    
                    if haber_data:
                        self.csv_kaydet(haber_data)
                        toplam_basarili += 1
                    else:
                        print("Haber atlandı: İçerik çekilemedi")
                        toplam_basarisiz += 1
                    
                    time.sleep(1)
                
                time.sleep(2)
                
            print(f"\nTüm işlem tamamlandı:")
            print(f"Toplam başarılı: {toplam_basarili}")
            print(f"Toplam başarısız: {toplam_basarisiz}")
            
        except Exception as e:
            print(f"Bir hata oluştu: {str(e)}")
        finally:
            self.driver.quit()

if _name_ == "_main_":
    scraper = KriptoHaberScraper()
    scraper.calistir()

Kripto haberleri ana sayfası açılıyor...
Toplam 1 sayfa bulundu.

Sayfa 1/1 işleniyor...
Sayfa 1: 25 haber bulundu.

Sayfa 1 - Haber 1/25 - İşleniyor: https://haber.doviz.com/kripto-para-haberleri/rusya-yuksek-elektrik-tuketimi-nedeniyle-bazi-bolgelerde-kripto-madenciligini-yasaklayacak/750862
Haber kaydedildi: Rusya, yüksek elektrik tüketimi nedeniyle bazı bölgelerde kripto madenciliğini yasaklayacak

Sayfa 1 - Haber 2/25 - İşleniyor: https://haber.doviz.com/kripto-para-haberleri/tether-den-turk-yetkililere-bor-madenlerini-temsil-edecek-token-onerisi/750433
Haber kaydedildi: Tether’den Türk yetkililere bor madenlerini temsil edecek token önerisi

Sayfa 1 - Haber 3/25 - İşleniyor: https://haber.doviz.com/kripto-para-haberleri/garanti-bbva-kripto-ripple-ve-ibm-ile-is-birligi-yapti/750220
Haber kaydedildi: Garanti BBVA Kripto, Ripple ve IBM ile iş birliği yaptı

Sayfa 1 - Haber 4/25 - İşleniyor: https://haber.doviz.com/kripto-para-haberleri/bitcoin-son-uc-ayin-zirvesine-cikti-yukselis-su